# Sketchy

Purpose: This notebook provides a walk through the process of retrieving photos of the same class as a hand-drawn sketch with the model proposed in [Doodle to Search: Practical Zero-Shot Sketch-based Image Retrieval](http://dagapp.cvc.uab.es/doodle2search/CVPR2019.pdf). 
Complementary information and studies on the model can be found in [here](http://dagapp.cvc.uab.es/doodle2search/CVPR2019_Supplementery.pdf).

Contrarily to the paper above, we perform here a non-zero shot inference, which means that the classes in training, validation and testing are the same (only sketches and images are different). Moreover, we did not implement all the parts associated with the semantic loss. Further explanation are èrovided in the model description part.

## Package Import

In [1]:
#!git clone https://https://github.com/VisiumCH/AMLD-2021-Sketchy.git

#%cd AMLD-2021-Sketchy/notebooks/
%pwd

'/home/pauline.maury/AMLD-2021-Sketchy/notebooks/workshop'

In [2]:
# Python imports
import matplotlib.pyplot as plt
import numpy as np

# Codebase imports


## Model description

### Quick overview

The photo retrieval task aims to return photos that are in the same class as sketches drawn by hand.

Therefore, two encoders are trained: one for the photos and one for the sketches. Each encoder maps its input (photo or sketch) to an embedding space E (with E=256 dimensions).

Then, the embeddings are ranked by similarity based on their euclidean distance in the embedding space and the most similar photos to the sketch  are retrieved.

<img src="images/graph.png">

### Problem Formulation
$C$: set of all possible categories

$X = \{x_i\}^N_{i=1}$: set of photos

$Y = \{y_i\}^M_{i=1}$: set of sketches

$l_x : X → C$ and $l_y : Y → C$ are the two labelling functions for photos and sketches respectively.

Such that given an input sketch an optimal ranking of gallery images can be obtained.

### Encoder Networks
Given a distance function d(·, ·), the aim of the framework is to learn two embedding functions $Ф : X → R^D$ and $Ψ : Y → R^D$, which respectively map the photo and sketch domain into a common embedding space.

Given two photos $x_1, x_2 ∈ X$ and a sketch $y ∈ Y$, it is expected that the embeddings fulfill the following condition:

$$\begin{align}
& d(Ф(x_1), Ψ(y)) < d(Ф(x2), Ψ(y)), \\
& when \quad l_x(x_1) = l_y(y) \\
& and \quad l_x(x_2) ≠ l_y(y) \\
\end{align}$$

meaning that there is a shorter distance when photos and sketch belong to the same class. Here, $d$ is the euclidean distance.

The embedding function $Ф(·)$ and $Ψ(·)$ are defined as two CNNs (VGG 16) with attention where the last fully-connected layer has been replaced to match the embedding size E. The attention mechanism helps the system
to localise the important features and is learned end-to-end with the rest of the network. The output of the attention module is computed by $f + f * att$.

### Learning Objectives



The learning objective of the framework combines two losses: the <i>Triplet Loss</i> and the <i>Domain Loss</i>.

$\{a, p, n\}$ 
where $a ∈ Y^s$, $p ∈ X^s$ and $n ∈ X^s$ are respectively the anchor, positive and negative samples during
the training and $l_x(p) = l_y(a)$ and $l_x(n) ≠ l_y(a)$.

#### Triplet loss: 
This loss aims to reduce the distance between embedded sketch and image if they belong to the same class and increase it if they belong to different classes.

Defining distance between samples as $δ_+ = ||Ψ(a) − Ф(p)||_2$ and $δ_- = ||Ψ(a) − Ф(n)||_2$ for the positive and negative samples respectively, then, the ranking loss for a particular triplet can be formulated as $λ(δ_+, δ_−) = max\{0, µ+δ_+ −δ_−\}$ where $µ > 0$ is a margin parameter. 

Batch-wise, the loss is defined as:
$$\begin{align}
& L_t = \sum_{i=1}^{N} λ(δ^i_+, δ^i_-) \\
\end{align}$$

 
the order aimed by this loss is $δ_− > δ_+ + µ$, if this is the case, the network is not updated, otherwise, the weights of the network are updated accordingly.

#### Domain loss:
This loss aims to explicitly enforce the mapping of sketch and image samples to a common space.

Given the embedding $Ф(·)$ and $Ψ(·)$, we make use of a Multilayer Perceptron (MLP) as a binary classifier trying to predict which was the initial domain. Purposefully, in order to create indistinguishable embedding we use a GRL defined as $R_λ(·)$, which applies the identity function during the forward pass $R_λ(x) = x$,
whereas during the backward pass it multiplies the gradients by the meta-parameter $−λ$, $\frac{dR_λ}{dx}= −λI$. This operation reverses the sign of the gradient that flows through the CNNs. 

Following the notation,
$f : R^D → [0, 1]$ be the MLP and $e ∈ R^D$ an embedding coming from the encoders network. Then we can define the binary cross entropy of one of the samples as $l_t(e) = tlog(f(R_{λ_d}(e))) + (1 − t) log(1 − f(R_{λ_d}(e)))$, where $e$ is the embedding obtained by the encoder network and $t$ is 0 and 1 for sketch and photo domains respectively.
Hence, the domain loss is defined as:

$$\begin{align}
& L_d = \frac{1}{3N}\sum_{i=1}^{N} ( l_0(Ψ(a_i)) + l_1(Ф(p_i)) + l_1(Ф(n_i)) ) \\
\end{align}$$

See the paper for additional information.